In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from scipy import stats
import numpy as np
import sklearn
import statsmodels.api as sm
import os
from itertools import chain
import itertools
from sklearn.preprocessing import OneHotEncoder
from collections import ChainMap


%matplotlib inline

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings(action='once')

In [4]:
currentDirectory = os.getcwd()
parentDirectory = os.path.abspath(os.path.join(currentDirectory, os.pardir))
train_df = pd.read_csv(os.path.join(
   parentDirectory,
   "input/house-prices-advanced-regression-techniques/train.csv"))
test_df = pd.read_csv(os.path.join(
   parentDirectory,
   "input/house-prices-advanced-regression-techniques/test.csv"))

#dataset_df = dataset_df.drop('Id', axis=1)
dataset_df = pd.concat([train_df, test_df], sort=False, ignore_index=True)
#test_Id = test_df['Id']
#StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
#                            'Survived': predictions })
#StackingSubmission.to_csv("submission.csv", index=False)

In [5]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

- MSSubClass: The building class
- MSZoning: The general zoning classification
- LotFrontage: Linear feet of street connected to property
- LotArea: Lot size in square feet
- Street: Type of road access
- Alley: Type of alley access
- LotShape: General shape of property
- LandContour: Flatness of the property
- Utilities: Type of utilities available
- LotConfig:  Lot configuration
- LandSlope: Slope of property
- Neighborhood: Physical locations within Ames city limits
- Condition1: Proximity to main road or railroad
- Condition2: Proximity to main road or railroad (if a second is present)
- BldgType: Type of dwelling
- HouseStyle: Style of dwelling
- OverallQual: Overall material and finish quality
- OverallCond:  Overall condition rating
- YearBuilt: Original construction date
- YearRemodAdd: Remodel date
- RoofStyle: Type of roof
- RoofMatl: Roof material
- Exterior1st: Exterior covering on house
- Exterior2nd: Exterior covering on house (if more than one material)
- MasVnrType: Masonry veneer type
- MasVnrArea: Masonry veneer area in square feet
- ExterQual: Exterior material quality
- ExterCond: Present condition of the material on the exterior
- Foundation: Type of foundation
- BsmtQual: Height of the basement
- BsmtCond: General condition of the basement
- BsmtExposure: Walkout or garden level basement walls
- BsmtFinType1: Quality of basement finished area
- BsmtFinSF1: Type 1 finished square feet
- BsmtFinType2: Quality of second finished area (if present)
- BsmtFinSF2: Type 2 finished square feet
- BsmtUnfSF: Unfinished square feet of basement area
- TotalBsmtSF: Total square feet of basement area
- Heating: Type of heating
- HeatingQC: Heating quality and condition
- CentralAir: Central air conditioning
- Electrical: Electrical system
- 1stFlrSF:First Floor square feet
- 2ndFlrSF:Second floor square feet
- LowQualFinSF:Low quality finished square feet (all floors)
- GrLivArea:  Above grade (ground) living area square feet
- BsmtFullBath: Basement full bathrooms
- BsmtHalfBath: Basement half bathrooms
- FullBath:  Full bathrooms above grade
- HalfBath: Half bathrooms above grade
- BedroomAbvGr:  Number of bedrooms above basement level
- KitchenAbvGr: Number of kitchens
- KitchenQual: Kitchen quality
- TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
- Functional: Home functionality rating
- Fireplaces: Number of fireplaces
- FireplaceQu: Fireplace quality
- GarageType: Garage location
- GarageYrBlt: Year garage was built
- GarageFinish:  Interior finish of the garage
- GarageCars:  Size of garage in car capacity
- GarageArea:  Size of garage in square feet
- GarageQual: Garage quality
- GarageCond: Garage condition
- PavedDrive: Paved driveway
- WoodDeckSF: Wood deck area in square feet
- OpenPorchSF:  Open porch area in square feet
- EnclosedPorch: Enclosed porch area in square feet
- 3SsnPorch: Three season porch area in square feet
- ScreenPorch: Screen porch area in square feet
- PoolArea: Pool area in square feet
- PoolQC:Pool quality
- Fence: Fence quality
- MiscFeature: Miscellaneous feature not covered in other categories
- MiscVal: $Value of miscellaneous feature
- MoSold: Month Sold
- YrSold: Year Sold
- SaleType: Type of sale
- SaleCondition: Condition of sale
- SalePrice:  the property's sale price in dollars. This is the target variable that you're trying to predict.

In [6]:
def miss_data(data):
    # From: https://www.kaggle.com/code/amrmohamed319/housing-price-with-ames-data-new-insights-top-1
    Miss_df = ((data == 0).sum()).to_frame()
    Miss_df = Miss_df.rename(columns = {0: 'zeros'})
    Miss_df.index.name = 'Feature'
    Miss_df['np.nan'] = (data.isnull()).sum()
    Miss_df['None'] = (data == "None").sum()
    Miss_df['total'] = Miss_df['zeros'] + Miss_df['np.nan'] + Miss_df['None']
    Miss_df['Percent'] = 100*Miss_df['total']/len(data)
    Miss_df['Type'] = [data[i].dtype for i in Miss_df.index ]
    return    Miss_df.sort_values(ascending = False, by = 'Percent')

In [7]:
print(len(miss_data(dataset_df)))
miss_data(dataset_df)

81


,zeros,np.nan,None,total,Percent,Type
Feature,,,,,,
PoolQC,0,2909,0,2909,99.657417,object
PoolArea,2906,0,0,2906,99.554642,int64
3SsnPorch,2882,0,0,2882,98.732443,int64
LowQualFinSF,2879,0,0,2879,98.629668,int64
MiscVal,2816,0,0,2816,96.471394,int64
MiscFeature,0,2814,0,2814,96.402878,object
BsmtHalfBath,2742,2,0,2744,94.004796,float64
Alley,0,2721,0,2721,93.216855,object
ScreenPorch,2663,0,0,2663,91.229873,int64


In [8]:
def defineAsCategorical(dataset, column):
    dataset[column] = dataset[column].astype("category")
    return dataset

def mapToGrades(dataset, column):
    dataset[column] = dataset[column].map({
        "Ex":0,
        "Gd":1,
        "TA":2,
        "Fa":3,
        "Po":4
    }).fillna("NA")
    return dataset

def fillNanValues(dataset, column, valueToFillWith, fillNone = True):
    if fillNone:
        dataset[column].replace('None', np.nan, inplace=True)
    dataset[column] = dataset[column].fillna(valueToFillWith)
    return dataset

def getMedianValueForQuantile(groupByQuantileColumn, column, quantile):
    return (
        dataset_df.groupby(groupByQuantileColumn)[column].median()[quantile])

def fillValuePlausible(dataset, column, referenceColumn, quantiles = 10):
    """
    Column has Nan values. Indices with those Nan values can be grouped with other indices
    according to their value in a referenceColumn. Fill the Nan values in the column by
    writing in the median value of those elements that share the same quantile in the
    referenceColumn.
    """
    # creates another column in which the referenceColumn is split into quantiles
    dataset["quantileReferenceColumn"] = pd.qcut(dataset[referenceColumn],
        quantiles, duplicates = "drop", labels = False)
    # save indices in column for which nan values are stored
    indicesNanColumn = dataset[column][np.isnan(dataset[column])].index
    # rewrite the values in the column that has nan values with the median value 
    # of that column of the elements that are in the same quantile in the referenceColumn 
    dataset[column][indicesNanColumn] = [
        getMedianValueForQuantile("quantileReferenceColumn", column, quantile)
        for quantile in dataset["quantileReferenceColumn"][indicesNanColumn].tolist()]
    # remove unneeded column
    dataset.drop("quantileReferenceColumn", axis = 1)
    return dataset

def getColumnsWithStrings(dataset):
    columnsWithStrings = []
    for column in dataset_df.columns:
        booleanList_strings = [type(element)==str for element in dataset_df[column]]
        if True in booleanList_strings:
            columnsWithStrings.append(column)
    return columnsWithStrings

def getColumnsWithSpecificSubstrings(dataset, substring):
    columnsWithSubstrings = []
    for column in dataset_df.columns:
        if (substring in column) is True:
            columnsWithSubstrings.append(column)
    return columnsWithSubstrings

In [9]:
#MSSubClass
stories_1= [20, 30, 40, 85, 120]
stories_1andHalf = [45,50,150]
stories_2 = [60,70,90,160,190,]
stories_2andHalf = [75]
stories_3andAbove = [180, 80]
d = ChainMap(dict.fromkeys(stories_1, 1),
             dict.fromkeys(stories_1andHalf, 1.5),
             dict.fromkeys(stories_2, 2),
             dict.fromkeys(stories_2andHalf, 2.5),
             dict.fromkeys(stories_3andAbove, 3))
dataset_df['Stories'] = dataset_df['MSSubClass'].map(d.get)

#MSZoning
dataset_df["ResidentialDensity"] = dataset_df["MSZoning"].map({
    "RH":4,
    "RL":2,
    "RP":1,
    "RM":3
}).fillna(0)
dataset_df["Water"] = dataset_df["MSZoning"].map({
    "FV":1,
}).fillna(0)
dataset_df["Commercial"] = dataset_df["MSZoning"].map({
    "C":1,
}).fillna(0)
dataset_df["Industrial"] = dataset_df["MSZoning"].map({
    "I":1,
}).fillna(0)

#LotShape
dataset_df["PropertyIrregularity"] = dataset_df["LotShape"].map({
    "Reg":0,
    "IR1":1,
    "IR2":2,
    "IR3":3
}).fillna(0)

#LandContour
dataset_df["PropertyFlatness"] = dataset_df["LandContour"].map({
    "Lvl":1,
}).fillna(0)

#Utilities
dataset_df["UtilitiesGrade"] = dataset_df["Utilities"].map({
    "AllPub":3,
    "NoSewr":2,
    "NoSeWa":1,
    "ELO":0	
}).fillna(0)

#SlopeofProperty
dataset_df["PropertySlope"] = dataset_df["LandSlope"].map({
    "Gtl":0,
    "Mod":1,
    "Sev":2
}).fillna(0)

#Neighborhood, RoofStyle
dummycolumns = [
    "Neighborhood", "RoofStyle", "RoofMatl"]
#if all(dummycolumns in dataset_df.columns):
dataset_df = pd.get_dummies(dataset_df, columns=dummycolumns,
                                drop_first = True)
#dataset_df = pd.concat([dataset_df, dum.set_index(dataset_df.index)], axis=1) 

#Proximity
def proximityColumns(dataset):
    dataset_withConditions = dataset[["Condition1", "Condition2"]]
    dataset["distanceToImportantStreet"] = dataset.apply(
        lambda row: 0 if ("Artery" == row.Condition1 or "Artery" == row.Condition2)
                    else 1 if ("Feedr" == row.Condition1 or "Feedr" == row.Condition2)
                    else 2, axis = 1)
    dataset["distanceToRailroad"] = dataset.apply(
        lambda row: 0 if ("RRAn" in [row.Condition1, row.Condition2] or "RRAe" in [row.Condition1, row.Condition2])
                    else 1 if ("RRNn" in [row.Condition1, row.Condition2] or "RRNe" in [row.Condition1, row.Condition2])
                    else 2, axis = 1)
    dataset["distanceToFeature"] = dataset.apply(
        lambda row: 0 if ("PosA" == row.Condition1 or "PosA" == row.Condition2)
                    else 1 if ("PosN" == row.Condition1 or "PosN" == row.Condition2)
                    else 2, axis = 1)
    dataset["proximityNumber"] = dataset.apply(
        lambda row: 0 if ("Norm" == row.Condition1 and "Norm" == row.Condition2)
                    else 1 if ("Norm" in [row.Condition1, row.Condition2])
                    else 2, axis = 1)
    return dataset
dataset_df = proximityColumns(dataset_df)

#BldgType
#HouseStyle

#YearRemodAdd

#ExterQual

#Grades
for column in ["ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC", "KitchenQual", "FireplaceQu", "GarageQual", "GarageCond" ]:
    dataset_df = mapToGrades(dataset_df, column)

#MasVnrArea
dataset_df["groundFloorArea"] = dataset_df["GrLivArea"] + dataset_df["GarageArea"]
dataset_df = fillValuePlausible(dataset_df, column = "MasVnrArea", referenceColumn = "groundFloorArea", quantiles = 10)

#Electrical
dataset_df.dropna(subset=['Electrical'], how='any', axis = 0,  inplace=True)

for column in ["LotFrontage"]:
    dataset_df = fillNanValues(dataset_df, column, valueToFillWith = 0)

for column in ["Alley", "MasVnrType"]:
    dataset_df = fillNanValues(dataset_df, column, valueToFillWith = "NoneGiven")


for column in getColumnsWithStrings(dataset_df):
    dataset_df = defineAsCategorical(dataset_df, column)

basementColumns = getColumnsWithSpecificSubstrings(dataset = dataset_df, substring = "Bsmt")


/tmp/ipykernel_33/3111429447.py:39: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset[column][indicesNanColumn] = [
/tmp/ipykernel_33/3111429447.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [10]:
cols_to_drop = ['PoolQC', 'PoolArea', 'MiscFeature', 'MiscVal', 'Alley', 'Fence', 'WoodDeckSF',
            'FireplaceQu', 'Fireplaces', 'MasVnrType', 'MasVnrArea',
            'FullBath', 'HalfBath' , 'BsmtFullBath' ,'BsmtHalfBath',
            'BsmtFinSF1' , 'BsmtFinSF2' , 'BsmtUnfSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
            '1stFlrSF', '2ndFlrSF', 'LowQualFinSF']
dataset_df = dataset_df.drop(cols_to_drop, axis = 1)

In [32]:
categoricalAndObjectColumns = dataset_df.select_dtypes(["category", "object"]).columns.tolist()
dataset_df = dataset_df.drop(categoricalAndObjectColumns, axis = 1)

In [33]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2918 entries, 0 to 2918
Data columns (total 73 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Id                         2918 non-null   int64  
 1   MSSubClass                 2918 non-null   int64  
 2   LotFrontage                2918 non-null   float64
 3   LotArea                    2918 non-null   int64  
 4   OverallQual                2918 non-null   int64  
 5   OverallCond                2918 non-null   int64  
 6   YearBuilt                  2918 non-null   int64  
 7   YearRemodAdd               2918 non-null   int64  
 8   ExterQual                  2918 non-null   int64  
 9   ExterCond                  2918 non-null   int64  
 10  TotalBsmtSF                2917 non-null   float64
 11  HeatingQC                  2918 non-null   int64  
 12  GrLivArea                  2918 non-null   int64  
 13  BedroomAbvGr               2918 non-null   int64  
 1

In [35]:
### Define the features with which the model shall not train
#input_features = dataset_df.columns.tolist()
#for feature in ["Id", "MSSubClass", "MSZoning"]:
  #  input_features.remove(feature)

In [36]:
train_df_full = dataset_df[~dataset_df["SalePrice"].isnull()]
train_df_features = train_df_full.drop(['SalePrice'], axis=1)
train_df_featuresArray = train_df_features.values
train_df_outcome = train_df_full['SalePrice'].ravel()

test_df_full = dataset_df[dataset_df["SalePrice"].isnull()]
test_df_features = test_df_full.drop(['SalePrice'], axis=1)
test_df_featruesArray = test_df_features.values
test_df_outcome = test_df_full['SalePrice'].ravel()

/tmp/ipykernel_33/201191592.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  train_df_outcome = train_df_full['SalePrice'].ravel()
/tmp/ipykernel_33/201191592.py:9: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  test_df_outcome = test_df_full['SalePrice'].ravel()


#### Train model

In [37]:
import category_encoders as ce

cols_CE = train_df.select_dtypes(include= 'object').columns
cols_CE

Index([], dtype='object')

In [38]:
cbe = ce.cat_boost.CatBoostEncoder(a = 1)
cbe.fit(train_df[cols_CE], train_df['SalePrice'])
train_df[cols_CE] = cbe.transform(train_df[cols_CE])
train_df[cols_CE] = cbe.transform(train_df[cols_CE])

In [39]:
train_df.head(4)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,180921.19589,65.0,8450,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,7,5,2003,2003,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,196.0,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,706,180921.19589,0,150,856,180921.19589,180921.19589,180921.19589,180921.19589,856,854,0,1710,1,0,2,1,3,1,180921.19589,8,180921.19589,0,180921.19589,180921.19589,2003.0,180921.19589,2,548,180921.19589,180921.19589,180921.19589,0,61,0,0,0,0,180921.19589,180921.19589,180921.19589,0,2,2008,180921.19589,180921.19589,208500
1,2,20,180921.19589,80.0,9600,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,6,8,1976,1976,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,0.0,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,978,180921.19589,0,284,1262,180921.19589,180921.19589,180921.19589,180921.19589,1262,0,0,1262,0,1,2,0,3,1,180921.19589,6,180921.19589,1,180921.19589,180921.19589,1976.0,180921.19589,2,460,180921.19589,180921.19589,180921.19589,298,0,0,0,0,0,180921.19589,180921.19589,180921.19589,0,5,2007,180921.19589,180921.19589,181500
2,3,60,180921.19589,68.0,11250,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,7,5,2001,2002,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,162.0,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,486,180921.19589,0,434,920,180921.19589,180921.19589,180921.19589,180921.19589,920,866,0,1786,1,0,2,1,3,1,180921.19589,6,180921.19589,1,180921.19589,180921.19589,2001.0,180921.19589,2,608,180921.19589,180921.19589,180921.19589,0,42,0,0,0,0,180921.19589,180921.19589,180921.19589,0,9,2008,180921.19589,180921.19589,223500
3,4,70,180921.19589,60.0,9550,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,7,5,1915,1970,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,0.0,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,180921.19589,216,180921.19589,0,540,756,180921.19589,180921.19589,180921.19589,180921.19589,961,756,0,1717,1,0,1,0,3,1,180921.19589,7,180921.19589,1,180921.19589,180921.19589,1998.0,180921.19589,3,642,180921.19589,180921.19589,180921.19589,0,35,272,0,0,0,180921.19589,180921.19589,180921.19589,0,2,2006,180921.19589,180921.19589,140000


In [40]:
from xgboost import XGBRegressor

In [41]:
model_xgb = XGBRegressor(
                        n_estimators = 1000,
                        max_depth = 8,
                        subsample = 0.6,
                        #min_child_weight = 60, 
                        learning_rate = 0.1)

In [42]:
model_xgb.fit(train_df.drop(['Id', 'SalePrice'], axis =1), train_df['SalePrice'],
            verbose = False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [43]:
y_test = model_xgb.predict(test_df)
y_test

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:MSZoning: object, Street: object, Alley: object, LotShape: object, LandContour: object, Utilities: object, LotConfig: object, LandSlope: object, Neighborhood: object, Condition1: object, Condition2: object, BldgType: object, HouseStyle: object, RoofStyle: object, RoofMatl: object, Exterior1st: object, Exterior2nd: object, MasVnrType: object, ExterQual: object, ExterCond: object, Foundation: object, BsmtQual: object, BsmtCond: object, BsmtExposure: object, BsmtFinType1: object, BsmtFinType2: object, Heating: object, HeatingQC: object, CentralAir: object, Electrical: object, KitchenQual: object, Functional: object, FireplaceQu: object, GarageType: object, GarageFinish: object, GarageQual: object, GarageCond: object, PavedDrive: object, PoolQC: object, Fence: object, MiscFeature: object

In [ ]:
sub = pd.DataFrame({'Id': range(1461,2920), 'SalePrice': np.exp(y_test)})
sub = sub.set_index('Id')
sub.to_csv('submission.csv')